In [1]:
import numpy as np
import pandas as pd
from qiskit import Aer, QuantumCircuit, transpile, execute
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

# Correct path to the uploaded file
file_path = 'epsx1_real.csv'  # Use the actual uploaded file ID path

# Read the Excel file
data = pd.read_csv(file_path)

# Display the first few rows to confirm successful loading
print(data.head)
print(data.size)

<bound method NDFrame.head of       Photon Energy  Absorption
0              0.70        7.24
1              0.71        7.24
2              0.71        7.25
3              0.71        7.25
4              0.71        7.26
...             ...         ...
1995           5.69       -9.52
1996           5.69       -9.48
1997           5.70       -9.44
1998           5.70       -9.39
1999           5.70       -9.35

[2000 rows x 2 columns]>
4000


In [3]:
# Assuming the last column is the target variable
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels

In [4]:
# Split dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# Define the QNN circuit
def create_qnn_circuit(params):
    circuit = QuantumCircuit(2)
    circuit.h([0, 1])  # Initialize in superposition
    circuit.rx(params[0], 0)  # Apply RX rotation
    circuit.ry(params[1], 1)  # Apply RY rotation
    circuit.cx(0, 1)  # CNOT gate
    return circuit

In [7]:
# Execute the circuit and get the output state
def execute_circuit(circuit):
    backend = Aer.get_backend('statevector_simulator')
    transpiled_circuit = transpile(circuit, backend)
    job = execute(transpiled_circuit, backend)
    result = job.result()
    statevector = result.get_statevector()
    return statevector

In [8]:
# Basic gradient descent optimizer for QNN
def train_qnn(X_train, y_train, epochs=100):
    params = np.random.rand(2)  # Initialize parameters
    learning_rate = 0.01

    for epoch in range(epochs):
        for i in range(len(X_train)):
            circuit = create_qnn_circuit(params)
            statevector = execute_circuit(circuit)

            # Get the output and compute the loss (mean squared error)
            output = np.abs(statevector[1]) ** 2  # Probability of the second basis state
            target = y_train[i]  # True label
            loss = (output - target) ** 2

            # Gradient descent update
            params -= learning_rate * (output - target) * np.array([np.cos(params[0]), np.sin(params[1])])

        if epoch % 10 == 0:
            print(f'Gradient Descent Epoch {epoch}, Loss: {loss:.4f}')

    return params

In [9]:
# Shukla-Vedula optimizer for QNN
def shukla_vedula_optimization(X_train, y_train, epochs=100):
    params = np.random.rand(2)  # Initialize parameters
    learning_rate = 0.01

    for epoch in range(epochs):
        for i in range(len(X_train)):
            circuit = create_qnn_circuit(params)
            statevector = execute_circuit(circuit)

            # Get the output and compute the loss (mean squared error)
            output = np.abs(statevector[1]) ** 2  # Probability of the second basis state
            target = y_train[i]  # True label
            loss = (output - target) ** 2

            # Shukla-Vedula Update: Gradient-free adjustment
            params += learning_rate * (target - output) * np.array([np.sin(params[0]), np.cos(params[1])])

        if epoch % 10 == 0:
            print(f'Shukla-Vedula Epoch {epoch}, Loss: {loss:.4f}')

    return params

In [10]:
# Train and test the QNN with Gradient Descent
print("Training with Gradient Descent Optimizer:")
gd_params = train_qnn(X_train, y_train)

Training with Gradient Descent Optimizer:
Gradient Descent Epoch 0, Loss: 642.1156
Gradient Descent Epoch 10, Loss: 642.1156
Gradient Descent Epoch 20, Loss: 642.1156
Gradient Descent Epoch 30, Loss: 642.1156
Gradient Descent Epoch 40, Loss: 642.1156
Gradient Descent Epoch 50, Loss: 642.1156
Gradient Descent Epoch 60, Loss: 642.1156
Gradient Descent Epoch 70, Loss: 642.1156
Gradient Descent Epoch 80, Loss: 642.1156
Gradient Descent Epoch 90, Loss: 642.1156


In [11]:
print("\nTraining with Shukla-Vedula Optimizer:")
sv_params = shukla_vedula_optimization(X_train, y_train)


Training with Shukla-Vedula Optimizer:
Shukla-Vedula Epoch 0, Loss: 629.5081
Shukla-Vedula Epoch 10, Loss: 629.5081
Shukla-Vedula Epoch 20, Loss: 629.5081
Shukla-Vedula Epoch 30, Loss: 629.5081
Shukla-Vedula Epoch 40, Loss: 629.5081
Shukla-Vedula Epoch 50, Loss: 629.5081
Shukla-Vedula Epoch 60, Loss: 629.5081
Shukla-Vedula Epoch 70, Loss: 629.5081
Shukla-Vedula Epoch 80, Loss: 629.5081
Shukla-Vedula Epoch 90, Loss: 629.5081


In [15]:
# Test the QNN on the test set
def test_qnn(X_test, y_test, trained_params):
    correct_predictions = 0

    for i in range(len(X_test)):
        circuit = create_qnn_circuit(trained_params)
        statevector = execute_circuit(circuit)
        output = np.abs(statevector[1]) ** 2  # Probability of the second basis state
        prediction = int(output > 0.5)  # Threshold for binary classification

        if prediction == y_test[i]:
            correct_predictions += 1

    accuracy = correct_predictions / len(y_test)
    print(f'Accuracy: {accuracy:.2f}')

In [16]:
# Test the QNNs
print("\nTesting Gradient Descent QNN:")
test_qnn(X_test, y_test, gd_params)

print("\nTesting Shukla-Vedula QNN:")
test_qnn(X_test, y_test, sv_params)


Testing Gradient Descent QNN:
Accuracy: 0.00

Testing Shukla-Vedula QNN:
Accuracy: 0.00


In [19]:
# Plotting the dataset (optional, for 2D features only)
if X.shape[1] == 2:
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
    plt.title('Data Points')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()